Import Necessary Dependencies

---

In [1]:
import pandas as pd 

SAVE_DATA_PATH = "../../data/processed/preprocessed_movielens_data.csv"

Loading our data

---

In [2]:
BASE_DATA_PATH = "../../data/raw/movielens-data/"
MOVIES_DATA_PATH = BASE_DATA_PATH + "movies.csv"
LINKS_DATA_PATH = BASE_DATA_PATH + "links.csv"
RATINGS_DATA_PATH = BASE_DATA_PATH + "ratings.csv"

In [3]:
data_movies = pd.read_csv(MOVIES_DATA_PATH)
data_links = pd.read_csv(LINKS_DATA_PATH)
data_ratings = pd.read_csv(RATINGS_DATA_PATH)

print("Movies data : \n", data_movies.head())
print("Links data : \n", data_links.head())
print("Ratings data : \n", data_ratings.head())

Movies data : 
    movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
Links data : 
    movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0
Ratings data : 
    userId  movieId  rating   timestamp
0       1        1     4.0  1225734739
1       1      110     4.0  1225865086
2       1      158     4.0  1225733503
3       1      260     4.5  12257

1. Movies Data

---

In [4]:
data_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
rows, cols = data_movies.shape
print("Number of movies: ", rows)
print("Number of attributes per movie: ", cols)

Number of movies:  86537
Number of attributes per movie:  3


In [6]:
duplicate_titles = (data_movies['title']
    .value_counts()
    .reset_index()
)
duplicate_titles.columns = ['title', 'count']
duplicate_titles = duplicate_titles[duplicate_titles['count'] > 1]
number_of_duplicate_titles = duplicate_titles.shape[0]
print("Number of duplicate movie titles: ", number_of_duplicate_titles)

Number of duplicate movie titles:  202


In [7]:
data_movies = data_movies.drop_duplicates(subset="title")
data_movies.reset_index(drop=True, inplace=True)

In [8]:
rows, cols = data_movies.shape
print("Number of movies: ", rows)
print("Number of attributes per movie: ", cols)

Number of movies:  86330
Number of attributes per movie:  3


In [9]:
# Extract title and release year
data_movies[["title", "release_year"]] = data_movies["title"].str.extract(
    r"^(.*)\s\((\d{4})\)$"
)
# Convert year to integer
data_movies["release_year"] = data_movies["release_year"].astype("Int64")

In [10]:
data_movies.head()

,movieId,title,genres,release_year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


2. Ratings data

---

In [11]:
data_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


In [12]:
rows, cols = data_ratings.shape
print("Number of ratings: ", rows)
print("Number of attributes per rating: ", cols)

Number of ratings:  33832162
Number of attributes per rating:  4


In [13]:
data_ratings.drop(columns=["timestamp"], inplace=True)

In [14]:
count_users = data_ratings['userId'].nunique()
count_movies = data_ratings['movieId'].nunique()
print("Number of unique users: ", count_users)
print("Number of unique movies rated: ", count_movies)

Number of unique users:  330975
Number of unique movies rated:  83239


In [15]:
ratings_counts = data_ratings['rating'].value_counts().sort_index()
print("Ratings distribution: \n", ratings_counts)

Ratings distribution: 
 rating
0.5     566306
1.0    1013645
1.5     562409
2.0    2146492
2.5    1760733
3.0    6400664
3.5    4465001
4.0    8835955
4.5    3123055
5.0    4957902
Name: count, dtype: int64


In [16]:
data_negative_ratings = data_ratings[data_ratings['rating'] < 3.0]
data_positive_ratings = data_ratings[data_ratings['rating'] >= 4.0]

In [17]:
num_positive_ratings = data_positive_ratings.shape[0]
num_negative_ratings = data_negative_ratings.shape[0]
print("Number of positive ratings (>=4.0): ", num_positive_ratings)
print("Number of negative ratings (<3.0): ", num_negative_ratings)

Number of positive ratings (>=4.0):  16916912
Number of negative ratings (<3.0):  6049585


In [18]:
data_positive_ratings = data_positive_ratings.copy()
data_negative_ratings = data_negative_ratings.copy()
data_positive_ratings.drop(columns=['rating'], inplace=True)
data_negative_ratings.drop(columns=["rating"], inplace=True)

In [19]:
data_positive_ratings.head()

,userId,movieId
0,1,1
1,1,110
2,1,158
3,1,260
4,1,356


In [20]:
positive_users_data = (
    data_positive_ratings.groupby("movieId")["userId"]
    .apply(list)
    .reset_index(name="positive_users")
)

negative_users_data = (
    data_negative_ratings.groupby("movieId")["userId"]
    .apply(list)
    .reset_index(name="negative_users")
)

print("Positive users per movie: \n", positive_users_data.head())
print("Negative users per movie: \n", negative_users_data.head())

Positive users per movie: 
    movieId                                     positive_users
0        1  [1, 2, 7, 12, 24, 35, 42, 51, 54, 64, 72, 79, ...
1        2  [9, 41, 51, 73, 82, 101, 117, 177, 207, 210, 2...
2        3  [9, 41, 200, 314, 367, 473, 475, 540, 775, 940...
3        4  [260, 934, 1760, 2051, 2077, 2721, 2722, 2957,...
4        5  [88, 196, 207, 214, 323, 385, 464, 473, 503, 5...
Negative users per movie: 
    movieId                                     negative_users
0        1  [14, 87, 180, 187, 196, 339, 468, 479, 486, 49...
1        2  [14, 39, 50, 72, 79, 141, 148, 149, 227, 265, ...
2        3  [73, 149, 190, 285, 477, 562, 677, 889, 974, 1...
3        4  [347, 1677, 1980, 1997, 2334, 4305, 4730, 4733...
4        5  [56, 149, 189, 198, 285, 388, 456, 562, 589, 6...


In [21]:
positive_users_data["positive_count"] = positive_users_data["positive_users"].apply(len)
negative_users_data["negative_count"] = negative_users_data["negative_users"].apply(len)
print("Positive users with counts: \n", positive_users_data.head())
print("Negative users with counts: \n", negative_users_data.head())

Positive users with counts: 
    movieId                                     positive_users  positive_count
0        1  [1, 2, 7, 12, 24, 35, 42, 51, 54, 64, 72, 79, ...           50572
1        2  [9, 41, 51, 73, 82, 101, 117, 177, 207, 210, 2...           10622
2        3  [9, 41, 200, 314, 367, 473, 475, 540, 775, 940...            5152
3        4  [260, 934, 1760, 2051, 2077, 2721, 2722, 2957,...             781
4        5  [88, 196, 207, 214, 323, 385, 464, 473, 503, 5...            4407
Negative users with counts: 
    movieId                                     negative_users  negative_count
0        1  [14, 87, 180, 187, 196, 339, 468, 479, 486, 49...            6299
1        2  [14, 39, 50, 72, 79, 141, 148, 149, 227, 265, ...            6539
2        3  [73, 149, 190, 285, 477, 562, 677, 889, 974, 1...            3574
3        4  [347, 1677, 1980, 1997, 2334, 4305, 4730, 4733...            1025
4        5  [56, 149, 189, 198, 285, 388, 456, 562, 589, 6...            3864


In [22]:
# Sets of movieIds
all_movies = set(data_movies["movieId"])
positive_movies = set(positive_users_data["movieId"])
negative_movies = set(negative_users_data["movieId"])

# Counts
only_positive = len(positive_movies - negative_movies)
only_negative = len(negative_movies - positive_movies)
both = len(positive_movies & negative_movies)
total_positive = len(positive_movies)
total_negative = len(negative_movies)

# Movies with at least one rating (either positive or negative)
either = positive_movies | negative_movies
# Movies with no ratings (neither positive nor negative)
neither = all_movies - either

print("Number of movies with only positive ratings:", only_positive)
print("Number of movies with only negative ratings:", only_negative)
print("Number of movies with both positive and negative ratings:", both)
print("Total movies with positive ratings:", total_positive)
print("Total movies with negative ratings:", total_negative)

print("Number of movies with either positive or negative ratings:", len(either))
print("Number of movies with neither positive nor negative ratings:", len(neither))

Number of movies with only positive ratings: 10879
Number of movies with only negative ratings: 19140
Number of movies with both positive and negative ratings: 44120
Total movies with positive ratings: 54999
Total movies with negative ratings: 63260
Number of movies with either positive or negative ratings: 74139
Number of movies with neither positive nor negative ratings: 12364


In [23]:
data_ratings = pd.merge(
    positive_users_data,
    negative_users_data,
    on="movieId",
    how="outer",  # use 'outer' to keep movies that exist in either dataset
)
print(data_ratings.head())

   movieId                                     positive_users  positive_count  \
0        1  [1, 2, 7, 12, 24, 35, 42, 51, 54, 64, 72, 79, ...         50572.0   
1        2  [9, 41, 51, 73, 82, 101, 117, 177, 207, 210, 2...         10622.0   
2        3  [9, 41, 200, 314, 367, 473, 475, 540, 775, 940...          5152.0   
3        4  [260, 934, 1760, 2051, 2077, 2721, 2722, 2957,...           781.0   
4        5  [88, 196, 207, 214, 323, 385, 464, 473, 503, 5...          4407.0   

                                      negative_users  negative_count  
0  [14, 87, 180, 187, 196, 339, 468, 479, 486, 49...          6299.0  
1  [14, 39, 50, 72, 79, 141, 148, 149, 227, 265, ...          6539.0  
2  [73, 149, 190, 285, 477, 562, 677, 889, 974, 1...          3574.0  
3  [347, 1677, 1980, 1997, 2334, 4305, 4730, 4733...          1025.0  
4  [56, 149, 189, 198, 285, 388, 456, 562, 589, 6...          3864.0  


In [24]:
data_ratings.isnull().sum()

movieId               0
positive_users    19140
positive_count    19140
negative_users    10879
negative_count    10879
dtype: int64

In [25]:
# Replace NaN in user lists with empty lists
data_ratings["positive_users"] = data_ratings["positive_users"].apply(
    lambda x: x if isinstance(x, list) else []
)
data_ratings["negative_users"] = data_ratings["negative_users"].apply(
    lambda x: x if isinstance(x, list) else []
)

# Set counts to 0
data_ratings["positive_count"] = data_ratings["positive_count"].fillna(0).astype(int)
data_ratings["negative_count"] = data_ratings["negative_count"].fillna(0).astype(int)

data_ratings.isnull().sum()

movieId           0
positive_users    0
positive_count    0
negative_users    0
negative_count    0
dtype: int64

In [26]:
data_ratings.head()

,movieId,positive_users,positive_count,negative_users,negative_count
0,1,"[1, 2, 7, 12, 24, 35, 42, 51, 54, 64, 72, 79, ...",50572,"[14, 87, 180, 187, 196, 339, 468, 479, 486, 49...",6299
1,2,"[9, 41, 51, 73, 82, 101, 117, 177, 207, 210, 2...",10622,"[14, 39, 50, 72, 79, 141, 148, 149, 227, 265, ...",6539
2,3,"[9, 41, 200, 314, 367, 473, 475, 540, 775, 940...",5152,"[73, 149, 190, 285, 477, 562, 677, 889, 974, 1...",3574
3,4,"[260, 934, 1760, 2051, 2077, 2721, 2722, 2957,...",781,"[347, 1677, 1980, 1997, 2334, 4305, 4730, 4733...",1025
4,5,"[88, 196, 207, 214, 323, 385, 464, 473, 503, 5...",4407,"[56, 149, 189, 198, 285, 388, 456, 562, 589, 6...",3864


In [27]:
data_ratings['movieId'].nunique()

74139

3. Links data

---

In [28]:
data_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [29]:
rows, cols = data_links.shape
print("Number of link entries: ", rows)
print("Number of attributes per link entry: ", cols)

Number of link entries:  86537
Number of attributes per link entry:  3


In [30]:
data_links.isnull().sum()

movieId      0
imdbId       0
tmdbId     126
dtype: int64

In [31]:
data_links.dropna(inplace=True)
data_links.isnull().sum()

movieId    0
imdbId     0
tmdbId     0
dtype: int64

In [32]:
common_movies = set(data_ratings["movieId"]) & set(data_links["movieId"]) & set(data_movies["movieId"])
common = len(common_movies)
print("Number of movies common between ratings data and links data:", common)

Number of movies common between ratings data and links data: 73859


In [33]:
# Filter all three dataframes
data_ratings = data_ratings[data_ratings["movieId"].isin(common_movies)]
data_links = data_links[data_links["movieId"].isin(common_movies)]
data_movies = data_movies[data_movies["movieId"].isin(common_movies)]

Merge Dataframes into a single dataframe

---

In [34]:
print("Movies data \n", data_movies.head())
print("Links data \n", data_links.head())

Movies data 
    movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  release_year  
0  Adventure|Animation|Children|Comedy|Fantasy          1995  
1                   Adventure|Children|Fantasy          1995  
2                               Comedy|Romance          1995  
3                         Comedy|Drama|Romance          1995  
4                                       Comedy          1995  
Links data 
    movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0


In [35]:
merged_data = pd.merge(data_movies, data_links, on="movieId", how="inner")
print(merged_data.head())

   movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  release_year  imdbId   tmdbId  
0  Adventure|Animation|Children|Comedy|Fantasy          1995  114709    862.0  
1                   Adventure|Children|Fantasy          1995  113497   8844.0  
2                               Comedy|Romance          1995  113228  15602.0  
3                         Comedy|Drama|Romance          1995  114885  31357.0  
4                                       Comedy          1995  113041  11862.0  


In [36]:
final_merged_data = pd.merge(merged_data, data_ratings, on="movieId", how="inner")
print(final_merged_data.head())

   movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  release_year  imdbId   tmdbId  \
0  Adventure|Animation|Children|Comedy|Fantasy          1995  114709    862.0   
1                   Adventure|Children|Fantasy          1995  113497   8844.0   
2                               Comedy|Romance          1995  113228  15602.0   
3                         Comedy|Drama|Romance          1995  114885  31357.0   
4                                       Comedy          1995  113041  11862.0   

                                      positive_users  positive_count  \
0  [1, 2, 7, 12, 24, 35, 42, 51, 54, 64, 72, 79, ...           50572   
1  [9, 41, 51, 73, 82, 101, 117, 177, 207, 210, 2...           10622   
2  [9, 41, 200, 314, 367, 473, 475, 54

Final Data Transformation

---

In [37]:
final_merged_data = final_merged_data.drop(columns=['movieId'])
print(final_merged_data.head())

                         title                                       genres  \
0                    Toy Story  Adventure|Animation|Children|Comedy|Fantasy   
1                      Jumanji                   Adventure|Children|Fantasy   
2             Grumpier Old Men                               Comedy|Romance   
3            Waiting to Exhale                         Comedy|Drama|Romance   
4  Father of the Bride Part II                                       Comedy   

   release_year  imdbId   tmdbId  \
0          1995  114709    862.0   
1          1995  113497   8844.0   
2          1995  113228  15602.0   
3          1995  114885  31357.0   
4          1995  113041  11862.0   

                                      positive_users  positive_count  \
0  [1, 2, 7, 12, 24, 35, 42, 51, 54, 64, 72, 79, ...           50572   
1  [9, 41, 51, 73, 82, 101, 117, 177, 207, 210, 2...           10622   
2  [9, 41, 200, 314, 367, 473, 475, 540, 775, 940...            5152   
3  [260, 934, 1760, 

In [38]:
# Convert genres string to list of lowercase genres
final_merged_data["genres"] = final_merged_data["genres"].apply(
    lambda x: [genre.strip().lower() for genre in x.split("|")]
)

# Convert numeric imdbId to string with 'tt' prefix
final_merged_data["imdbId"] = final_merged_data["imdbId"].apply(
    lambda x: f"tt{int(x):07d}"
)

# create movie_id by combining tmdbId and imdbId
final_merged_data["tmdbId"] = final_merged_data["tmdbId"].astype(int)
final_merged_data["movie_id"] = final_merged_data["tmdbId"].astype(str) + "_" + final_merged_data["imdbId"]
final_merged_data.drop(columns=["tmdbId", "imdbId"], inplace=True)
print(final_merged_data.head())

                         title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                              genres  release_year  \
0  [adventure, animation, children, comedy, fantasy]          1995   
1                     [adventure, children, fantasy]          1995   
2                                  [comedy, romance]          1995   
3                           [comedy, drama, romance]          1995   
4                                           [comedy]          1995   

                                      positive_users  positive_count  \
0  [1, 2, 7, 12, 24, 35, 42, 51, 54, 64, 72, 79, ...           50572   
1  [9, 41, 51, 73, 82, 101, 117, 177, 207, 210, 2...           10622   
2  [9, 41, 200, 314, 367, 473, 475, 540, 775, 940...            5152   
3  [260, 934, 1760, 2051, 2077, 2721, 2722, 2957,...             781   
4  [88, 196, 2

Save the final merged data

---

In [39]:
final_merged_data.to_csv(SAVE_DATA_PATH, index=False)